In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

def preprocess_image(img_path):
    """Loads and preprocesses an image."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data

def extract_features(img_path):
    """Extracts features from an image using ResNet50."""
    img_data = preprocess_image(img_path)
    features = model.predict(img_data)
    return features

def compute_similarity(img_path1, img_path2):
    """Computes similarity between two images."""
    features1 = extract_features(img_path1)
    features2 = extract_features(img_path2)
    similarity = cosine_similarity(features1, features2)
    return similarity[0][0]

# Example usage
img_path1 = '/content/necklace1.jpg'
img_path2 = '/content/necklace2.jpg'

similarity_score = compute_similarity(img_path1, img_path2)
print(f"Similarity score between the images: {similarity_score}")
if similarity_score > 0.8:
    print("Images are similar.")
else:
    print("Images are not similar.")


1/1 [==============================] - 0s 198ms/step
Similarity score between the images: 0.9002416133880615
Images are similar.


In [3]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [4]:
def preprocess_image(img_path):
    """Loads and preprocesses an image."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data

def extract_features(img_path):
    """Extracts features from an image using ResNet50."""
    img_data = preprocess_image(img_path)
    features = model.predict(img_data)
    return features

def compute_similarity(img_path1, img_paths):
    """Computes similarity between an image and multiple images."""
    features1 = extract_features(img_path1)

    similarities = []
    for img_path2 in img_paths:
        features2 = extract_features(img_path2)
        similarity = cosine_similarity(features1, features2)
        similarities.append(similarity)

    return similarities

In [10]:
# Example usage
img_path1 = '/content/necklace1.jpg'
img_paths = [
    '/content/necklace2.jpg',
    '/content/Screenshot 2024-06-15 180558.jpg',
    '/content/Screenshot 2024-06-15 180645.jpg',
    '/content/Screenshot 2024-06-15 180653.jpg',
    '/content/Screenshot 2024-06-15 180701.jpg',
    '/content/Screenshot 2024-06-15 180711.jpg',
    '/content/Screenshot 2024-06-15 180721.jpg'


]

similarities = compute_similarity(img_path1, img_paths)

threshold = 0.8  # hyperparameter
similar_images = []
for i, similarity_score in enumerate(similarities):
    if similarity_score > threshold:
        print(f"Image {img_paths[i]} is similar with score: {similarity_score}")
        similar_images.append(img_paths[i])

if similar_images:
    print("Similar images found:", similar_images)
else:
    print("No similar images found.")

1/1 [==============================] - 0s 192ms/step
Image /content/necklace2.jpg is similar with score: [[0.9002416]]
Similar images found: ['/content/necklace2.jpg']


In [14]:
from flask import Flask, request, jsonify
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import boto3
from botocore.exceptions import NoCredentialsError



In [12]:
pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.0 MB/s eta 0:00:00


In [ ]:
app = Flask(__name__)

# AWS S3 configuration
S3_BUCKET = 'cfg-products'
S3_REGION = 'ap-south-1'  # e.g., 'us-west-1'
s3_client = boto3.client('s3', region_name=S3_REGION)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

def preprocess_image(img_path):
    """Loads and preprocesses an image."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data

def extract_features(img_path):
    """Extracts features from an image using ResNet50."""
    img_data = preprocess_image(img_path)
    features = model.predict(img_data)
    return features

def list_s3_images(bucket):
    """Lists all image files in the specified S3 bucket."""
    response = s3_client.list_objects_v2(Bucket=bucket)
    if 'Contents' in response:
        return [item['Key'] for item in response['Contents'] if item['Key'].lower().endswith(('png', 'jpg', 'jpeg'))]
    return []

def download_s3_image(bucket, key, download_path):
    """Downloads an image from S3."""
    s3_client.download_file(bucket, key, download_path)

def compute_similarity(features1, img_paths):
    """Computes similarity between a feature vector and features of multiple images."""
    similarities = []
    for img_path in img_paths:
        features2 = extract_features(img_path)
        similarity = cosine_similarity(features1, features2)
        similarities.append(similarity[0][0])
    return similarities

@app.route('/compare', methods=['POST'])
def compare_images():
    if 'image' not in request.files:
        return jsonify({"error": "Please provide an image to compare."}), 400

    image_file = request.files['image']
    img_path = os.path.join('/tmp', image_file.filename)
    image_file.save(img_path)

    # Upload the image to S3
    s3_client.upload_file(img_path, S3_BUCKET, image_file.filename)

    # Extract features of the uploaded image
    features1 = extract_features(img_path)

    # List and download all images from the S3 bucket
    s3_images = list_s3_images(S3_BUCKET)
    similarities = []
    for s3_image in s3_images:
        download_path = os.path.join('/tmp', os.path.basename(s3_image))
        download_s3_image(S3_BUCKET, s3_image, download_path)
        features2 = extract_features(download_path)
        similarity = cosine_similarity(features1, features2)
        similarities.append((s3_image, similarity[0][0]))

    # Determine similar images
    threshold = 0.8  # hyperparameter
    similar_images = [img for img, score in similarities if score > threshold]

    if similar_images:
        return jsonify({"similar_images": similar_images})
    else:
        return jsonify({"message": "No similar images found."})

if __name__ == '__main__':
    app.run(debug=True)